In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/lib/kaggle/gcp.py
/kaggle/input/scl-2021-ds/sampleSubmission.csv
/kaggle/input/scl-2021-ds/train.csv
/kaggle/input/scl-2021-ds/test.csv
/kaggle/working/__notebook_source__.ipynb


In [ ]:
data = pd.read_csv('/kaggle/input/scl-2021-ds/train.csv')
data

In [4]:
data["POI"] = data["POI/street"].str.split(pat='/').apply(lambda x:x[0])
data["Street"] = data["POI/street"].str.split(pat='/').apply(lambda x:x[1])


In [5]:
data

,id,raw_address,POI/street,POI,Street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/,,
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung
3,3,"toko dita, kertosono",toko dita/,toko dita,
4,4,jl. orde baru,/jl. orde baru,,jl. orde baru
...,...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,,jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",/raya cila kko,,raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,/,,
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,taman asri,


In [6]:
TRAIN_DATA = []
STREET_DATA = set()
POI_DATA = set()

In [7]:
for index,row in data.iterrows():
    if (index+1) % 10000 == 0:
        print(index+1)
    entities = []
    if row["POI"] != '':
        start = 0
        end = 0
        find = row["raw_address"].find(row["POI"])
        if find == -1:
            sp = row["POI"].split(' ')
            count = 0
            for i in sp:
                v = i if len(i) < 3 else i[0:3]
                find = row["raw_address"].find(v)
                if find != -1:
                    if count == 0:
                        start = find
                    end = find
                    for j in range(len(i)):
                        if (find+j) < len(row["raw_address"]) and row["raw_address"][find+j] == i[j]:
                            end = find+j
                        else:
                            break
                    count+=1
        else:
            start = find
            end = find+len(row["POI"]) - 1
        POI_DATA.add(row["POI"])
        entities.append((start,end+1,row["POI"]))
    if row["Street"] != '':
        start = 0
        end = 0
        find = row["raw_address"].find(row["Street"])
        if find == -1:
            sp = row["Street"].split(' ')
            count = 0
            for i in sp:
                v = i if len(i) < 3 else i[0:3]
                find = row["raw_address"].find(v)
                if find != -1:
                    if count == 0:
                        start = find
                    end = find
                    for j in range(len(i)):
                        if (find+j) < len(row["raw_address"]) and row["raw_address"][find+j] == i[j]:
                            end = find+j
                        else:
                            break
                    count+=1
        else:
            start = find
            end = find+len(row["Street"]) - 1
        entities.append((start,end+1,row["Street"]))
        STREET_DATA.add(row["Street"])
    TRAIN_DATA.append((row["raw_address"],{'entities':entities}))

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000


In [8]:
TRAIN_DATA

[('jl kapuk timur delta sili iii lippo cika 11 a cicau cikarang pusat',
  {'entities': [(0, 40, 'jl kapuk timur delta sili iii lippo cika')]}),
 ('aye, jati sampurna', {'entities': []}),
 ('setu siung 119 rt 5 1 13880 cipayung', {'entities': [(5, 10, 'siung')]}),
 ('toko dita, kertosono', {'entities': [(0, 9, 'toko dita')]}),
 ('jl. orde baru', {'entities': [(0, 13, 'jl. orde baru')]}),
 ('raya samb gede, 299 toko bb kids',
  {'entities': [(20, 32, 'toko bb kids'), (0, 14, 'raya samb gede')]}),
 ('kem mel raya, no 4 bojong rawalumbu rt 1 36 rawalumbu',
  {'entities': [(0, 12, 'kem mel raya')]}),
 ('tela keuramat kuta alam', {'entities': [(0, 4, 'tela')]}),
 ('gg. i wates magersari', {'entities': [(0, 5, 'gg. i')]}),
 ('bunga ncole ix 2', {'entities': [(0, 14, 'bunga ncole ix')]}),
 ('cikahuripan sd neg boj 02 klap boj, no 5 16877',
  {'entities': [(12, 25, 'sd negeri bojong 02'), (26, 34, 'klap boj')]}),
 ('yaya atohar,', {'entities': [(0, 11, 'yayasan atohariyah')]}),
 ('abim ix 24 5'

In [9]:
POI_DATA

{'marlina',
 'pelabuhan padang bai',
 'sma kristen kristen penabur kota tangerang',
 'mujiari bengkel',
 'sukemi',
 'masjid ad-dakwah',
 'pt. kencana indah bersama',
 'kompl griya muti timur 1',
 'rolak outbound kids surabaya',
 'tpa ruhui rahayu',
 'kebon ibu ieda',
 'pt. mitra aiko utama',
 'laboratorium klinik tanjung',
 'kaona muslim mart',
 'tk ar ridwan',
 'kantor desa kalisongo',
 'djambu indah',
 'christiana jewellery',
 'ikan bakar hidayah',
 'toko lasman',
 'kk phone',
 'musholla al-hikmah',
 'masjid sumpah terate udik',
 'smks al hamidiyah',
 'raden barbershop',
 'carlam',
 'duta anaiah',
 'mechanic online mbl',
 'kebab afwan',
 'apotek antang farma makassar',
 'bengkel rental mobil 99',
 'cv. hingdi oto kencana',
 'ksp bhayangkara kediri - jatim',
 'rumah makan padang uni',
 'warung sate barokah pak zen',
 'jala seafood (jls)',
 'masjid halal',
 'dinas pertambangan dan energi',
 'obat kutil kelamin  ubat perontok kutil kondiloma ampuh  penawar kemaluan',
 'mie kriting dahsy

In [10]:
STREET_DATA

{'jl. pasir impun',
 'punai 3',
 'amsar',
 'a y',
 'haji chai',
 'tungu asri',
 'gg. sd',
 'btn mak baji',
 'jl. cemp. putih bar. xi jl. d. baru',
 'komp paku',
 'indu xii',
 'katra',
 'jl. raya jati asih',
 'gun jati v',
 'jl. gar g',
 'moha a wahab',
 'situ bung',
 'gang tahir',
 'agung per 13',
 'alf',
 'suka - lemb',
 'ri r t si',
 'jl. kapten maulana lubis',
 'h marj',
 'raya ban ten',
 'jl.raya desa modong',
 'perta a',
 'jl. nangka ii',
 'g. pay iii',
 'komp perum pln',
 'danau indah 4',
 'lintas sumatera-s',
 'raya sading-meawi',
 'jl. kena',
 'raya bulua',
 'c. 8',
 'besar sipi',
 'jl. duri sela iv',
 'jln. padat karya',
 'gg. setra asih',
 '14 febr',
 'serdang',
 'nak kota kepah',
 'ant raya depan swal misi pasa',
 'pec ii',
 'kemba utara',
 'tebet mas indah 1',
 'c. teluk gong',
 'jend ahmad yani timur',
 'senam vii',
 'poros amol',
 'jln. lagundri',
 'jl. h. sai haji sai',
 'dun',
 'taman sent blok k',
 'kopo cirang',
 'mr. teuku muha hasan',
 'jl lase',
 'gg. h. sabe ubah'

In [12]:
n_iter=5

In [13]:
import spacy
from tqdm import tqdm

In [14]:
spacy.require_gpu()

True

In [15]:
ner_model = spacy.blank('en')
print("Create Blank en model")

Create Blank en model


In [16]:
if 'ner' not in ner_model.pipe_names:
    ner = ner_model.create_pipe('ner')
    ner_model.add_pipe(ner, last=True)
else:
    ner = ner_model.get_pipe('ner')

In [19]:
count = 0
for _,annotations in TRAIN_DATA:
    count += 1
    if count % 10000 == 0:
        print(count)
    all_entities = annotations.get('entities')
    for ent in all_entities:
        ner.add_label(ent[2])
other_pipes = [pipe for pipe in ner_model.pipe_names if pipe != 'ner']

10000
20000
30000
40000
50000


KeyboardInterrupt: 

In [ ]:
with ner_model.disable_pipes(*other_pipes):
    optimizer = ner_model.begin_training()
    for itn in range(n_iter):
        losses = {}
        print(n_iter+1)
        count = 0
        for text,annotations in tqdm(TRAIN_DATA):
            count += 1
            if count % 10000 == 0:
                print(count)
            ner_model.update([text],[annotations],drop=0.5,sgd=optimizer,losses=losses)

/opt/conda/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  **kwargs
/opt/conda/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  **kwargs


In [ ]:
test_df = pd.read_csv('/kaggle/input/scl-2021-ds/test.csv')

In [ ]:
test_df

In [ ]:
TEST_RESULT = []

In [ ]:
for index,row in test_df.iterrows():
    doc = ner_model(row["raw_address"])
    poiTest = ''
    streetTest =''
    for ent in doc.ents:
        temp = ent.label_
        if temp in POI_DATA:
            poiTest = ent.text
        elif temp in STREET_DATA:
            streetTest = ent.text
    TEST_RESULT.append([row["id"],poiTest+'/'+streetTest])

In [ ]:
answerDF = pd.DataFrame(TEST_RESULT,columns=['id', 'POI/street'])

In [ ]:
answerDF.to_csv('/output/kaggle/working/output.csv',index=False)